In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/Thesis/ASD

/content/gdrive/MyDrive/Thesis/ASD


In [ ]:
cd /content/gdrive/MyDrive/Thesis/Thesis/ASD

/content/gdrive/.shortcut-targets-by-id/1eA5gTf1FtgP2bh0p-wWtNYDI9KsYbaiU/Thesis/ASD


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import functools
import numpy.ma as ma # for masked arrays
import glob
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from itertools import groupby
import sklearn

In [4]:
!pip install captum

     |████████████████████████████████| 1.4 MB 4.1 MB/s 


In [5]:
from sklearn.model_selection import train_test_split
from captum.attr import IntegratedGradients
from captum.attr import Saliency
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
from captum.attr import Saliency
import torchvision

### Helper functions for computing correlations

In [6]:
def get_corr_data(df):
              
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()
        
def get_corr_matrix(filename,data_path):
    # returns correlation matrix
    for file in os.listdir(data_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

## Loading the data 

In [7]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [8]:
data_df = pd.read_csv('./Phenotypes/Phenotypic_V1_0b_preprocessed949.csv', encoding= 'unicode_escape')
data_df.DX_GROUP = data_df.DX_GROUP.map({1: 1, 2:0})

data_path = './Datasets/CPAC/rois_cc200/'
data_df['FILE_PATH'] = data_df['FILE_ID'].apply(lambda x : os.path.join(data_path,x + '_rois_cc200.1D')) 

print('Length of data frame : ', len(data_df))
print(data_df.head())

Length of data frame :  949
   Unnamed: 0  SUB_ID  X  subject SITE_ID       FILE_ID  DX_GROUP  DSM_IV_TR  \
0           0   50003  2    50003    PITT  Pitt_0050003         1          1   
1           1   50004  3    50004    PITT  Pitt_0050004         1          1   
2           2   50006  5    50006    PITT  Pitt_0050006         1          1   
3           3   50007  6    50007    PITT  Pitt_0050007         1          1   
4           4   50009  8    50009    PITT  Pitt_0050009         1          1   

   AGE_AT_SCAN  SEX  ... qc_anat_rater_2  qc_anat_notes_rater_2  \
0        24.45    1  ...              OK                    NaN   
1        19.09    1  ...              OK                    NaN   
2        13.37    1  ...              OK                    NaN   
3        17.78    1  ...              OK                    NaN   
4        33.86    1  ...              OK                    NaN   

   qc_func_rater_2       qc_func_notes_rater_2  qc_anat_rater_3  \
0               OK   

In [9]:
fpaths = data_df['FILE_PATH'].values
print('Number of Subjects available: ', len(fpaths))

Number of Subjects available:  949


In [ ]:
all_corr = {}

for i,path in enumerate(fpaths):

    # Extracting SFC
    fname = path.split('/')[-1]
    key = fname.split('_rois')[0]
    x = np.loadtxt(path)
    x = np.array(x, dtype = 'float32')
    corr = get_corr_data(x)
    # Extracting corresponding labels
    y = data_df[data_df['FILE_ID'] == key]['DX_GROUP'].item()

    all_corr[key] = (corr,y)

print('Length of correlations vector : ', len(all_corr))
pickle.dump(all_corr, open('./data/SFC_CC200.pkl', 'wb'))

Length of correlations vector :  949


In [10]:
all_corr = pickle.load(open('./data/SFC_CC200.pkl', 'rb'))
flist = np.array(list(all_corr.keys()))
labels = np.array([all_corr[f][1] for f in flist], dtype = 'uint8')
print('Length of Input subjects : ', len(flist))
print('Length of Output subjects : ', len(labels))
print('Distribution of Labels : ', np.unique(labels, return_counts = True))

Length of Input subjects :  949
Length of Output subjects :  949
Distribution of Labels :  (array([0, 1], dtype=uint8), array([530, 419]))


# DataLoader

In [11]:
class ASDDataset(Dataset):
    def __init__(self, all_corr, subjects):
        self.corr = all_corr
        self.subjects = subjects
        pass
    def __getitem__(self,idx):
        return torch.tensor(self.corr[self.subjects[idx]][0],dtype=torch.float),torch.tensor(self.corr[self.subjects[idx]][1],dtype=torch.float)
        pass
    def __len__(self):
        return len(self.subjects)
        pass

# Network 

In [12]:
# Auto Encoder and Classifier
class Network(nn.Module):
    def __init__(self, num_inputs = 19900):
        super(Network, self).__init__()
        
        self.num_inputs = num_inputs
        
        self.fc_encoder = nn.Sequential (
                nn.Linear(self.num_inputs,2048),
                nn.Tanh(),
                nn.Linear(2048,512),
                nn.Tanh())
        
        self.fc_decoder = nn.Sequential (
                nn.Linear(512,2048),
                nn.Tanh(),
                nn.Linear(2048,self.num_inputs),
                nn.Tanh())
         
        self.classifier = nn.Sequential (
            nn.Dropout(p=0.25),
            nn.Linear(512, 1),
        )

        self.sigmoid = nn.Sigmoid()           
         
    def forward(self, x, eval_classifier = True):

        x = self.fc_encoder(x)
        if eval_classifier:
            x_logit = self.classifier(x)   #   .squeeze(1)
            # x_logit = self.sigmoid(x_logit)
            return x_logit 

        x = self.fc_decoder(x)        
        return x

# Defining training and testing functions

In [13]:
def train(model, criterion, data_loader, mode='clf'):
    model.train()
    clf_loss = []
    ae_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for (inputs, targets) in data_loader :
        if len(inputs) != batch_size:          
            continue

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        inputs.requires_grad_(True)
        inputs.retain_grad()

        if mode == 'ae':        
            reconstructed = model(inputs, False)
            loss_ae = criterion(reconstructed, inputs) / len(inputs)              
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            ae_loss.append(loss_ae_np)           

        elif mode == 'clf':
            logits = model(inputs, True)
            logits = np.squeeze(logits, 1)
            loss_clf = criterion(logits, targets)
            logits = torch.sigmoid(logits)
            proba = logits.detach().cpu().numpy()
            predictions = np.round(proba)           
            final_targets.append(targets.detach().cpu().numpy())
            final_predictions.append(predictions)
            
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()           
            clf_loss.append(loss_clf_np)
            
        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return np.mean(clf_loss), train_accuracy
    else:
        return np.mean(ae_loss), None


def validate(model, criterion, data_loader, mode='clf'):
    model.eval()
    clf_loss = []
    ae_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for (inputs, targets) in data_loader :
        if len(inputs) != batch_size:           
            continue

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        inputs.requires_grad_(True)
        inputs.retain_grad()

        if mode == 'ae':        
            reconstructed = model(inputs, False)
            loss_ae = criterion(reconstructed, inputs) / len(inputs)              
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            ae_loss.append(loss_ae_np)           

        elif mode == 'clf':
            logits = model(inputs, True)
            logits = np.squeeze(logits, 1)
            loss_clf = criterion(logits, targets)
            logits = torch.sigmoid(logits)
            proba = logits.detach().cpu().numpy()
            predictions = np.round(proba)           
            final_targets.append(targets.detach().cpu().numpy())
            final_predictions.append(predictions)
            
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()           
            clf_loss.append(loss_clf_np)
            
        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return np.mean(clf_loss), train_accuracy
    else:
        return np.mean(ae_loss), None

def test(model, criterion, data_loader, mode = 'clf'):

    clf_loss = []
    ae_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    with torch.no_grad():
        model.eval()
        for (inputs, targets) in data_loader :
            if len(inputs) != batch_size:           
                continue

            inputs, targets = inputs.to(device), targets.to(device)
            if mode == 'ae':        
                reconstructed = model(inputs, False)
                loss_ae = criterion_ae(reconstructed, inputs) / len(inputs)           
                loss_total = loss_ae
                loss_ae_np = loss_ae.detach().cpu().numpy()
                ae_loss.append(loss_ae_np)           

            if mode == 'clf':
                logits = model(inputs, True)
                logits = np.squeeze(logits, 1)
                loss_clf = criterion_clf(logits, targets)
                logits = torch.sigmoid(logits)
                proba = logits.detach().cpu().numpy()
                predictions = np.round(proba)           
                final_targets.append(targets.detach().cpu().numpy())
                final_predictions.append(predictions)
                
                loss_total = loss_clf
                loss_clf_np = loss_clf.detach().cpu().numpy()           
                clf_loss.append(loss_clf_np)
              
    
    if (final_targets is  None) and (final_predictions is None):
        return np.mean(ae_loss), None
    
    final_targets = np.concatenate(final_targets)
    final_predictions = np.concatenate(final_predictions)
    mlp_acc, mlp_sens, mlp_spef = confusion(final_targets, final_predictions)
    metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                    'senstivity' : np.round(mlp_sens,4), 
                    'specificity' : np.round(mlp_spef,4), 
                    'loss' : np.round(np.mean(clf_loss),4)}               
    return  metrics_dict  

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
def attribute_image_features(algorithm, inputs):
    model.zero_grad()
    model.eval()
    tensor_attributions = algorithm.attribute(inputs = inputs, target = 0, return_convergence_delta=True)  
    return tensor_attributions

# ASD 2 Layer Model Training

In [16]:
# Define Parameters
p_fold = 10
batch_size = 16
lr_ae, lr_clf = 0.0001, 0.0001
ae_epochs, clf_epochs = 50, 50     # 50, 50
weight_decay_ae, weight_decay_clf = 0.1, 0.1
n_inputs = 19900  # Automate this according to the atlas 

In [17]:
crossval_acc, crossval_sen, crossval_spec, crossval_loss, attributions = [], [], [], [], [] 
all_folds_splits = {}
kf = StratifiedKFold(n_splits = p_fold, random_state = 1, shuffle = True)

start = time.time()
for fold,(train_index, test_index) in enumerate(kf.split(flist, labels)):

    train_subjects, test_subjects = flist[train_index],flist[test_index]
    train_labels = labels[train_index]   
    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, 
                                                      test_size = 0.20, random_state = 42, stratify = train_labels)
    
    print('Number of train subjects : ', len(train_subjects))
    print('Number of val subjects : ', len(val_subjects))
    print('Number of test subjects : ', len(test_subjects))

    fold_splits_dict = {} 
    fold_splits_dict['train'] = train_subjects
    fold_splits_dict['val'] = val_subjects
    fold_splits_dict['test'] = test_subjects

    all_folds_splits[fold] = fold_splits_dict
    verbose = (True if (fold == 0) else False)
   
    train_dataset = ASDDataset(all_corr, train_subjects)
    val_dataset = ASDDataset(all_corr, val_subjects)
    test_dataset = ASDDataset(all_corr, test_subjects)
    
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)                           

    model = Network(num_inputs = n_inputs)
    model = model.to(device)

    criterion_ae = nn.MSELoss(reduction='sum')         
    optimizer = optim.Adam(model.parameters(), lr = lr_ae, weight_decay = weight_decay_ae)          
    best_ae_model, count = None, 1
    best_ae_loss = sys.float_info.max
    
    print("Auto Encoder training Started-----------")
    for epoch in range(1, ae_epochs+1):

        print(f'Epoch {epoch}/{ae_epochs}')
        ae_train_loss, _ = train(model, criterion_ae, train_dataloader, mode = 'ae')
        print(f'AE Train loss: {(ae_train_loss):.4f}')

        ae_val_loss, _ = validate(model, criterion_ae, val_dataloader, mode = 'ae')
        print(f'AE Val loss: {(ae_val_loss):.4f}')

        if(ae_val_loss < best_ae_loss):     # Early Stopping 
            best_ae_model = model
            best_ae_loss = ae_val_loss
            count = 1
        else:
            count += 1     
        if(count == 10):  # Criteria
            break
              
    best_clf_model, best_clf_acc, count = None, 0.0, 1
    model = best_ae_model
    criterion_clf = nn.BCEWithLogitsLoss()      
    optimizer = optim.Adam(model.parameters(), lr = lr_clf, weight_decay = weight_decay_clf)

    print("Classifier training Started-----------")
    for epoch in range(1, clf_epochs+1):

        print(f'Epoch {epoch}/{clf_epochs}')
        clf_train_loss, train_acc = train(model, criterion_clf, train_dataloader, mode='clf')
        print(f'CLF Train loss: {(clf_train_loss):.4f}, Train Accuracy: {(train_acc):.4f}')

        clf_val_loss, val_acc = validate(model, criterion_clf, val_dataloader, mode='clf')
        print(f'CLF Val loss: {(clf_val_loss):.4f}, Validation Accuracy: {(val_acc):.4f}')

        if(val_acc > best_clf_acc):    # Early Stopping Criteria
            best_clf_model = model
            best_clf_acc = val_acc
            count = 1
        else:
            count += 1
        if(count == 10):
            break        

    metrics_dict = test(best_clf_model, criterion_clf, test_dataloader, mode = 'clf')

    print(f'Fold {fold+1}/{p_fold}')
    print(f'{metrics_dict}')
    print("--------------------------------------------")
    
    torch.save(best_clf_model.state_dict(), f'./data/Weights/Fold_{fold+1}.pth')    # To save the weights
    print(f'Fold {fold+1} weights are saved')

    crossval_acc.append(metrics_dict['accuracy'])
    crossval_sen.append(metrics_dict['senstivity'])
    crossval_spec.append(metrics_dict['specificity'])
    crossval_loss.append(metrics_dict['loss'])

print(f'Average Value after 10 Folds')
print(f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}')
pickle.dump(all_folds_splits, open('./data/All_Folds_Subjects.pkl', 'wb'))
print(f'Total time taken : {time.time()-start}')

Number of train subjects :  683
Number of val subjects :  171
Number of test subjects :  95
Auto Encoder training Started-----------
Epoch 1/50
AE Train loss: 823.0101
AE Val loss: 711.7103
Epoch 2/50
AE Train loss: 670.3008
AE Val loss: 651.9384
Epoch 3/50
AE Train loss: 614.5040
AE Val loss: 604.3463
Epoch 4/50
AE Train loss: 571.0931
AE Val loss: 564.4747
Epoch 5/50
AE Train loss: 535.3869
AE Val loss: 531.1636
Epoch 6/50
AE Train loss: 504.8393
AE Val loss: 499.9503
Epoch 7/50
AE Train loss: 478.3432
AE Val loss: 473.2162
Epoch 8/50
AE Train loss: 453.2615
AE Val loss: 449.3344
Epoch 9/50
AE Train loss: 431.1058
AE Val loss: 427.9079
Epoch 10/50
AE Train loss: 410.7231
AE Val loss: 407.6306
Epoch 11/50
AE Train loss: 392.7664
AE Val loss: 388.7137
Epoch 12/50
AE Train loss: 377.8387
AE Val loss: 373.3183
Epoch 13/50
AE Train loss: 362.8664
AE Val loss: 358.2180
Epoch 14/50
AE Train loss: 347.3077
AE Val loss: 344.0820
Epoch 15/50
AE Train loss: 336.8014
AE Val loss: 331.3573
Epoch 

# End